In [1]:
import requests

# Testeamos la API que creamos
### Aprendizaje Automático II - CEIA - FIUBA 

En esta notebook testeamos la API del modelo que hicimos usando [FastAPI](https://fastapi.tiangolo.com/).

## main_1

Empezamos con la implementación más simple, debemos ejecutar:

```Bash
uvicorn main_1:app --host 0.0.0.0 --port 80
```

Usemos `requests` para obtener las respuesta del modelo, en esta implementación, los features van como parámetros de consulta:

In [2]:
endpoint_model = "http://localhost/predict/"
response = requests.post(endpoint_model)
response.text

'{"detail":[{"type":"missing","loc":["query","size"],"msg":"Field required","input":null},{"type":"missing","loc":["query","height"],"msg":"Field required","input":null},{"type":"missing","loc":["query","weight"],"msg":"Field required","input":null},{"type":"missing","loc":["query","number_of_whiskers"],"msg":"Field required","input":null}]}'

In [3]:
response.reason

'Unprocessable Entity'

Nos falta pasarle los features:

In [16]:
query_params = {
    "size": 10.5,
    "height": 20.3,
    "weight": 15.7,
    "number_of_whiskers": 8
}

response = requests.post(endpoint_model, params=query_params)
response

<Response [200]>

In [11]:
response.json()

{'prediction': 'Cat'}

## main_2

```Bash
uvicorn main_2:app --host 0.0.0.0 --port 80
```

En este caso, ahora los parámetros van en el cuerpo de la solicitud en un formato JSON. En general, es recomendable a los features del modelo que vayan en el cuerpo.

In [26]:
data = {
    "size": -10.5,
    "height": 20.3,
    "weight": 15.7,
    "number_of_whiskers": 8
}

response = requests.post(endpoint_model, json=data)
response

<Response [200]>

In [27]:
response.json()

{'prediction': 'Dog'}

## main_3

```Bash
uvicorn main_3:app --host 0.0.0.0 --port 80
```

Ahora usando Pydantic, le dimos restricciones para que no se pueda pasar cualquier valor

In [28]:
data = {
    "size": -10.5,
    "height": 20.3,
    "weight": 15.7,
    "number_of_whiskers": 8
}

response = requests.post(endpoint_model, json=data)
response

<Response [422]>

In [29]:
response.reason

'Unprocessable Entity'

In [30]:
response.json()

{'detail': [{'type': 'greater_than_equal',
   'loc': ['body', 'size'],
   'msg': 'Input should be greater than or equal to 0',
   'input': -10.5,
   'ctx': {'ge': 0.0}}]}

Ahora la API controla que los valores tengan un rango de valores apropiados.

In [31]:
data = {
    "size": 10.5,
    "height": 20.3,
    "weight": 15.7,
    "number_of_whiskers": 8
}

response = requests.post(endpoint_model, json=data)
response

<Response [200]>

In [32]:
response.json()

{'prediction': 'Cat'}

Ahora agregamos si hacemos GET en la raíz, nos devuelva información de la API

In [33]:
response = requests.get("http://localhost")
response.json()

{'name': 'ML Model API',
 'description': 'A simple machine learning model API for predicting cats or dogs.',
 'version': '1.0'}

## main_4

```Bash
uvicorn main_4:app --host 0.0.0.0 --port 80
```

Finalmente agregamos un método simple de autenticación:

In [34]:
data = {
    "size": 10.5,
    "height": 20.3,
    "weight": 15.7,
    "number_of_whiskers": 8
}

headers = {
    "api-key": "bad-key" 
}

response = requests.post(endpoint_model, json=data, headers=headers)
response

<Response [403]>

In [35]:
response.reason

'Forbidden'

In [36]:
response.json()

{'detail': 'Invalid API Key'}

In [37]:
data = {
    "size": 10.5,
    "height": 20.3,
    "weight": 15.7,
    "number_of_whiskers": 8
}

headers = {
    "api-key": "test-key" 
}

response = requests.post(endpoint_model, json=data, headers=headers)
response

<Response [200]>

In [38]:
response.reason

'OK'

In [39]:
response.json()

{'prediction': 'Cat'}

In [ ]:
response = requests.get("https://api.thecatapi.com/")
response.text

En este caso, al llamar a la URL base, aparece un mensaje genérico que dice `"The Cat API"`. Esto se debe a que se está llamando a la URL base, que normalmente se usa para información muy básica sobre una API, no para datos reales.

Llamar a la URL base por sí solo no es muy útil, pero ahí es donde los **endpoints** resultan útiles. Un **endpoint** es una parte de la URL que especifica qué recurso se desea obtener. Las API bien documentadas suelen contener una referencia de API, lo cual es prácticamente necesario para saber exactamente qué endpoints y recursos tiene una API y cómo usarlos. **Documenten sus APIs**

The API Cat nos presenta su [documentación](https://developers.thecatapi.com/). Ahí se encuentra el endpoint `/breeds` que se puede utilizar para recuperar todos los recursos de raza disponibles.

In [ ]:
response = requests.get("https://api.thecatapi.com/v1/breeds")
response.text

In [ ]:
response = requests.get("https://api.thecatapi.com/v1/breeds")
response

In [ ]:
response.status_code

In [ ]:
response.text

In [ ]:
response.headers

In [ ]:
response.request

In [ ]:
# Veamos el request
request = response.request

In [ ]:
request.url

In [ ]:
request.path_url

In [ ]:
request.method

In [ ]:
request.headers